<a href="https://colab.research.google.com/github/subhashpolisetti/AutoGluon_ML_End-to-End_Implementations_Part-2/blob/main/2_AutoGluon_Sentiment_Analysis_and_Sentence_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis and Sentence Similarity with AutoGluon**

This notebook demonstrates how to use **AutoGluon** for performing **sentiment analysis** and **sentence similarity prediction** tasks. It is designed as an end-to-end guide for building, training, evaluating, and visualizing text-based machine learning models.

---

## **1. Installation and Setup**
- Install required libraries such as `autogluon.multimodal`, PyTorch, and its dependencies.
- Configure the environment to ensure compatibility for model training and evaluation.

---

## **2. Sentiment Analysis**
### **Objective**:
Classify sentences into **positive** or **negative** sentiments.

### **Dataset**:
- **Source**: AutoGluon Text Dataset (SST).
- **Description**: Contains labeled sentences for binary classification.

### **Steps**:
1. **Load and Inspect Data**: Explore the training and testing datasets.
2. **Model Training**:
   - Use `MultiModalPredictor` for binary classification.
   - Train the model on a subset of the training dataset for demonstration purposes.
3. **Evaluation**:
   - Measure accuracy and F1-score on the test dataset.
4. **Prediction**:
   - Predict sentiments of new sentences.
   - Visualize the class probabilities of predictions.

---

## **3. Sentence Similarity Prediction**
### **Objective**:
Predict a similarity score between two sentences on a scale from **0** (completely dissimilar) to **5** (identical).

### **Dataset**:
- **Source**: Semantic Textual Similarity dataset.
- **Description**: Includes pairs of sentences with human-annotated similarity scores.

### **Steps**:
1. **Load Data**: Import the sentence pairs for training and testing.
2. **Model Training**:
   - Use `MultiModalPredictor` for regression tasks.
   - Train the model using a similarity dataset.
3. **Evaluation**:
   - Compute metrics such as RMSE, Pearson Correlation, and Spearman Rank Correlation.
4. **Prediction**:
   - Predict similarity scores for new sentence pairs.

---

## **4. Visualization and Embedding Analysis**
- **Embedding Extraction**: Extract text embeddings for the test dataset using the trained model.
- **Visualization**:
  - Apply TSNE for dimensionality reduction.
  - Visualize embeddings in a 2D space to understand the separation between labels or sentence similarities.

---

## **5. Saving and Loading Models**
- Save the trained models using AutoGluon’s `save()` method.
- Reload models for reuse and verification of predictions.

---

## **Key Features**:
1. End-to-end workflow for text-based machine learning using AutoGluon.
2. Includes both classification and regression examples.
3. Provides visualization of embeddings for deeper insights.
4. Simple and reproducible steps for building NLP models.

This notebook is ideal for learning or demonstrating sentiment analysis and sentence similarity prediction using a powerful and user-friendly framework like AutoGluon.


In [ ]:
# Install the AutoGluon multimodal library for text analysis and other dependencies.
!pip install autogluon.multimodal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtim

In [ ]:
%matplotlib inline

import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
np.random.seed(123)

In [ ]:
from autogluon.core.utils.loaders import load_pd
train_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/train.parquet')
test_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sst/dev.parquet')
subsample_size = 1000  # subsample data for faster demo, try setting this to larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head(10)

,sentence,label
43787,very pleasing at its best moments,1
16159,", american chai is enough to make you put away...",0
59015,too much like an infomercial for ram dass 's l...,0
5108,a stirring visual sequence,1
67052,cool visual backmasking,1
35938,hard ground,0
49879,"the striking , quietly vulnerable personality ...",1
51591,pan nalin 's exposition is beautiful and myste...,1
56780,wonderfully loopy,1
28518,"most beautiful , evocative",1


In [ ]:
!pip install torch==2.0.0+cu117 torchaudio==2.0.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 503.2 kB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/cu117/torchaudio-2.0.1%2Bcu117-cp310-cp310-linux_x86_64.whl (4.4 MB)
  Using cached https://download.pytorch.org/whl/triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires torch<2.4,>=2.2, but you have torch 2.0.0+cu117 which is incompatible.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 2.0.0+cu1

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torchvision 0.18.1
Uninstalling torchvision-0.18.1:
  Successfully uninstalled torchvision-0.18.1


In [ ]:
!pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
  Using cached https://download.pytorch.org/whl/cu117/torch-2.0.1%2Bcu117-cp310-cp310-linux_x86_64.whl (1843.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.0 MB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/cu117/torchaudio-2.0.1%2Bcu117-cp310-cp310-linux_x86_64.whl (4.4 MB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.0.1+cu117, torchaudio==2.0.1+cu117 and torchvision==0.15.2+cu117 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.0.1+cu117
    torchvision 0.15.2+cu117 depends on torch==2.0.1
    torchaudio 2.0.1+cu117 depends on torch==2.0.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pi

In [ ]:
import torch
print(torch.__version__)

2.0.0+cu117


In [ ]:
test_score = predictor.evaluate(test_data)
print(test_score)

In [ ]:
test_score = predictor.evaluate(test_data, metrics=['acc', 'f1'])
print(test_score)

In [ ]:
sentence1 = "it's a charming and often affecting journey."
sentence2 = "It's slow, very, very, very slow."
predictions = predictor.predict({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Sentiment":', predictions[0])
print('"Sentence":', sentence2, '"Predicted Sentiment":', predictions[1])

In [ ]:
probs = predictor.predict_proba({'sentence': [sentence1, sentence2]})
print('"Sentence":', sentence1, '"Predicted Class-Probabilities":', probs[0])
print('"Sentence":', sentence2, '"Predicted Class-Probabilities":', probs[1])

In [ ]:
test_predictions = predictor.predict(test_data)
test_predictions.head()

In [ ]:
loaded_predictor = MultiModalPredictor.load(model_path)
loaded_predictor.predict_proba({'sentence': [sentence1, sentence2]})

In [ ]:
new_model_path = f"./tmp/{uuid.uuid4().hex}-automm_sst"
loaded_predictor.save(new_model_path)
loaded_predictor2 = MultiModalPredictor.load(new_model_path)
loaded_predictor2.predict_proba({'sentence': [sentence1, sentence2]})

In [ ]:
embeddings = predictor.extract_embedding(test_data)
print(embeddings.shape)

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, random_state=123).fit_transform(embeddings)
for val, color in [(0, 'red'), (1, 'blue')]:
    idx = (test_data['label'].to_numpy() == val).nonzero()
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], c=color, label=f'label={val}')
plt.legend(loc='best')

In [ ]:
sts_train_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sts/train.parquet')[['sentence1', 'sentence2', 'score']]
sts_test_data = load_pd.load('https://autogluon-text.s3-accelerate.amazonaws.com/glue/sts/dev.parquet')[['sentence1', 'sentence2', 'score']]
sts_train_data.head(10)

In [ ]:
print('Min score=', min(sts_train_data['score']), ', Max score=', max(sts_train_data['score']))

In [ ]:
sts_model_path = f"./tmp/{uuid.uuid4().hex}-automm_sts"
predictor_sts = MultiModalPredictor(label='score', path=sts_model_path)
predictor_sts.fit(sts_train_data, time_limit=60)

In [ ]:
test_score = predictor_sts.evaluate(sts_test_data, metrics=['rmse', 'pearsonr', 'spearmanr'])
print('RMSE = {:.2f}'.format(test_score['rmse']))
print('PEARSONR = {:.4f}'.format(test_score['pearsonr']))
print('SPEARMANR = {:.4f}'.format(test_score['spearmanr']))

In [ ]:
sentences = ['The child is riding a horse.',
             'The young boy is riding a horse.',
             'The young man is riding a horse.',
             'The young man is riding a bicycle.']

score1 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[1]]}, as_pandas=False)

score2 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[2]]}, as_pandas=False)

score3 = predictor_sts.predict({'sentence1': [sentences[0]],
                                'sentence2': [sentences[3]]}, as_pandas=False)
print(score1, score2, score3)